## **Библиотеки, загрузка файла**

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
file_id = '1ATASmLwbd-sVPOJ7kChU9fumc15lRvwK'
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'
df = pd.read_csv(download_url)

df.head()

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,1,0


## **Предобработка данных**

Исходя из проделанного EDA принято решение:

* **Удалить** наблюдения **с годом рождения** клиента (Year_Birth) равным 1893, 1899 и 1900 как **аномально низкие**
* **Удалить** наблюдение **с доходом** (Income) равным **666.666 у.е., как аномально высокое**
* Заменить семейный статус (Marital_Status) с значений "Alone", "Abdsurd" и "Yolo" на "Single" как наиболее близкие по смысловому значению
* Так как в датасете **всего 24 пропущенных значений и они составляют лишь 1%** от наблюдений, **их удаление** при построении модели **не окажет значительного влияния** на результаты. Поэтому принято решение **удалить данные наблюдения** на следующих этапах анализа без их восстановления.


In [6]:
df = df[~df['Year_Birth'].isin([1893, 1899, 1900])]
df = df[df['Income'] != 666666]
df['Marital_Status'] = df['Marital_Status'].replace({'Alone': 'Single', 'Absurd': 'Single', 'Yolo': 'Single'})
df = df.dropna()


df = df.drop(['Dt_Customer', 'Id'], axis=1)
df = pd.get_dummies(df, columns=['Education', 'Marital_Status', 'Kidhome', 'Teenhome'], drop_first=True)

## **Построение и оценка модели**

Разделим данные на обучающую и тестовую выборки.

In [10]:
y = df['Response']
X = df.drop('Response', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Построим модель логистической регрессии

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

model_logreg=LogisticRegression(max_iter=1000, random_state=42)
model_logreg.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

Оценим ее по метрикам качества

In [17]:
y_pred_train = model_logreg.predict(X_train)
y_pred_prob_train = model_logreg.predict_proba(X_train)[:, 1]
y_pred_test = model_logreg.predict(X_test)
y_pred_prob_test = model_logreg.predict_proba(X_test)[:, 1]

metrics = {'Выборка': ['train', 'test'],
           'Accuracy': [accuracy_score(y_train, y_pred_train),accuracy_score(y_test, y_pred_test)],
           'Precision': [precision_score(y_train, y_pred_train),precision_score(y_test, y_pred_test)],
           'Recall': [recall_score(y_train, y_pred_train),recall_score(y_test, y_pred_test)],
           'F1-score': [f1_score(y_train, y_pred_train),f1_score(y_test, y_pred_test)],
           'ROC-AUC': [roc_auc_score(y_train, y_pred_prob_train),roc_auc_score(y_test, y_pred_prob_test)],
           'PR-AUC': [average_precision_score(y_train, y_pred_prob_train),average_precision_score(y_test, y_pred_prob_test)]}

pd.DataFrame(metrics)


,Выборка,Accuracy,Precision,Recall,F1-score,ROC-AUC,PR-AUC
0,train,0.854720,0.551724,0.180451,0.271955,0.801131,0.421212
1,test,0.875847,0.772727,0.253731,0.382022,0.853485,0.559923


Модель показывает хорошие результаты - она предсказывают 88% случаев верно на тестовой выборке. В то же время precision и recall подрос на тестовой выборке - это означает, что модель не переобучилась. Модель достаточно консервативна (recall < precision) и часто упускает купивших золотое членство, но хорошо различает классы.

## **Эксперимент 1. Масштабирование признаков**

In [21]:
from sklearn.preprocessing import StandardScaler
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

model_logreg_scaled=LogisticRegression(max_iter=1000, random_state=42)
model_logreg_scaled.fit(X_train_scaled, y_train)

y_pred_train_scaled= model_logreg_scaled.predict(X_train_scaled)
y_pred_prob_train_scaled =model_logreg_scaled.predict_proba(X_train_scaled)[:, 1]


y_pred_test_scaled = model_logreg_scaled.predict(X_test_scaled)
y_pred_prob_test_scaled = model_logreg_scaled.predict_proba(X_test_scaled)[:, 1]


metrics_scaled = {'Выборка': ['train', 'test'],
           'Accuracy': [accuracy_score(y_train, y_pred_train_scaled),accuracy_score(y_test, y_pred_test_scaled)],
           'Precision': [precision_score(y_train, y_pred_train_scaled),precision_score(y_test, y_pred_test_scaled)],
           'Recall': [recall_score(y_train, y_pred_train_scaled),recall_score(y_test, y_pred_test_scaled)],
           'F1-score': [f1_score(y_train, y_pred_train_scaled),f1_score(y_test, y_pred_test_scaled)],
           'ROC-AUC': [roc_auc_score(y_train, y_pred_prob_train_scaled),roc_auc_score(y_test, y_pred_prob_test_scaled)],
           'PR-AUC': [average_precision_score(y_train, y_pred_prob_train_scaled),average_precision_score(y_test, y_pred_prob_test_scaled)]}

pd.DataFrame(metrics_scaled)

,Выборка,Accuracy,Precision,Recall,F1-score,ROC-AUC,PR-AUC
0,train,0.872809,0.681416,0.289474,0.406332,0.845723,0.497462
1,test,0.873589,0.703704,0.283582,0.404255,0.862337,0.574051


С масштабирование признаков модель улучшила свои метрики качества - особенно precision, recall, f1 и AUC- метрики. Уменьшился разрыв между метриками на тесте и трейне - модель стала более стабильной, и переобучения также не случилось.

## **Эксперимент 2. Подбор гиперпараметров**
Попробуем найти лучшие гиперпараметры (C и penalty) для улучшения модели

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logreg_class = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
grid = GridSearchCV(logreg_class, param_grid, cv=5, scoring='f1')
grid.fit(X_train_scaled, y_train)

y_pred_train_grid = grid.predict(X_train_scaled)
y_pred_prob_train_grid = grid.predict_proba(X_train_scaled)[:, 1]

y_pred_test_grid = grid.predict(X_test_scaled)
y_pred_prob_test_grid = grid.predict_proba(X_test_scaled)[:, 1]

metrics_grid = {'Выборка': ['train', 'test'],
                'Accuracy': [accuracy_score(y_train, y_pred_train_grid),accuracy_score(y_test, y_pred_test_grid)],
                'Precision': [precision_score(y_train, y_pred_train_grid),precision_score(y_test, y_pred_test_grid)],
                'Recall': [recall_score(y_train, y_pred_train_grid),recall_score(y_test, y_pred_test_grid)],
                'F1-score': [f1_score(y_train, y_pred_train_grid),f1_score(y_test, y_pred_test_grid)],
                'ROC-AUC': [roc_auc_score(y_train, y_pred_prob_train_grid),roc_auc_score(y_test, y_pred_prob_test_grid)],
                'PR-AUC': [average_precision_score(y_train, y_pred_prob_train_grid),average_precision_score(y_test, y_pred_prob_test_grid)]}

print('Лучшие параметры:', grid.best_params_)
pd.DataFrame(metrics_grid)

Лучшие параметры: {'C': 100, 'penalty': 'l1'}


,Выборка,Accuracy,Precision,Recall,F1-score,ROC-AUC,PR-AUC
0,train,0.875071,0.692308,0.304511,0.422977,0.845918,0.496118
1,test,0.871332,0.692308,0.268657,0.387097,0.861782,0.571603


Модель при подборе гиперпараметров немного улучшила свои метрики качества на обучающей выборке, но ухудшила на тестовой в сравнении с моделью после масштабирования признаков. Это говорит нам о том, что масштабирование - привносит свой вклад в улучшение модели, а подбор гиперпараметров в данном случае таких результатов не принес (параметры слишком подстроились под обучающую выборку).

## **Эксперимент 3. RandomForest**

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train_scaled, y_train)

y_pred_train_rf = rf.predict(X_train_scaled)
y_pred_prob_train_rf = rf.predict_proba(X_train_scaled)[:, 1]

y_pred_test_rf = rf.predict(X_test_scaled)
y_pred_prob_test_rf = rf.predict_proba(X_test_scaled)[:, 1]


metrics_rf = {'Выборка': ['train', 'test'],
                'Accuracy': [accuracy_score(y_train, y_pred_train_rf),accuracy_score(y_test, y_pred_test_rf)],
                'Precision': [precision_score(y_train, y_pred_train_rf),precision_score(y_test, y_pred_test_rf)],
                'Recall': [recall_score(y_train, y_pred_train_rf),recall_score(y_test, y_pred_test_rf)],
                'F1-score': [f1_score(y_train, y_pred_train_rf),f1_score(y_test, y_pred_test_rf)],
                'ROC-AUC': [roc_auc_score(y_train, y_pred_prob_train_rf),roc_auc_score(y_test, y_pred_prob_test_rf)],
                'PR-AUC': [average_precision_score(y_train, y_pred_prob_train_rf),average_precision_score(y_test, y_pred_prob_test_rf)]}

pd.DataFrame(metrics_rf)

,Выборка,Accuracy,Precision,Recall,F1-score,ROC-AUC,PR-AUC
0,train,0.959299,0.994898,0.733083,0.844156,0.998581,0.991658
1,test,0.875847,0.772727,0.253731,0.382022,0.875675,0.597043


Модель сильно переобучилась на тренировочной выборке. В то же время на тесте - метрики качества не сильно отличаются от логистической регрессии после масштабирования признаков. Возросли ROC-AUC и PR-AUC метрики, а остальыне остались почти на том же уровне. Можно сделать вывод, что данный эксперимент сход по результатам с экспериментом 2 и заслуга здесь также в масштабировании признаков (так как мы обучали модель на отмасштабированных данных)

## **Эксперимент 4. Градиентный бустинг**

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, random_state=42)
gb.fit(X_train_scaled, y_train)

y_pred_train_gb = gb.predict(X_train_scaled)
y_pred_prob_train_gb = gb.predict_proba(X_train_scaled)[:, 1]

y_pred_test_gb = gb.predict(X_test_scaled)
y_pred_prob_test_gb = gb.predict_proba(X_test_scaled)[:, 1]


metrics_gb = {'Выборка': ['train', 'test'],
                'Accuracy': [accuracy_score(y_train, y_pred_train_gb),accuracy_score(y_test, y_pred_test_gb)],
                'Precision': [precision_score(y_train, y_pred_train_gb),precision_score(y_test, y_pred_test_gb)],
                'Recall': [recall_score(y_train, y_pred_train_gb),recall_score(y_test, y_pred_test_gb)],
                'F1-score': [f1_score(y_train, y_pred_train_gb),f1_score(y_test, y_pred_test_gb)],
                'ROC-AUC': [roc_auc_score(y_train, y_pred_prob_train_gb),roc_auc_score(y_test, y_pred_prob_test_gb)],
                'PR-AUC': [average_precision_score(y_train, y_pred_prob_train_gb),average_precision_score(y_test, y_pred_prob_test_gb)]}

pd.DataFrame(metrics_gb)

,Выборка,Accuracy,Precision,Recall,F1-score,ROC-AUC,PR-AUC
0,train,0.991521,1.000000,0.943609,0.970986,0.999714,0.998300
1,test,0.878104,0.658537,0.402985,0.500000,0.880279,0.600339


Градиентный бустинг имеет лучший результат на тестовой выборке по f1-score, recall и PR-AUC. Модель имеет более высокую полноту в сравнении с логистическими регрессями и случайным лесом. Но есть разрыв в метриках качества между тестом и трейном - это признак переобучения. В целом эта модель отработала лучше других в нашем случае - при дисбалансе классов.